In [1]:
import math
from pprint import pprint
from collections import Counter

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## ES2

### 1. Define a function that, given a text, creates the corresponding discrete source
### 2. Add a method Huffman to the class Source that computes the Huffman encoding.
### 3. Write a program that, given an input text, finds the Huffman encoding of the text. A decoding procedure to recover the original message by starting from the Huffman encoding is also required.

In [28]:
from queue import PriorityQueue


class Node(object):
    def __init__(self, symbol, prob, left=None, right=None):
        self.left = left
        self.right = right
        self.symbol = symbol
        self.prob = prob
        self.codeword = ""

    def __eq__(self, other):
        return self.prob == other.prob and self.symbol == other.symbol

    def __ne__(self, other):
        return self != other

    def __lt__(self, other):
        if self.prob == other.prob:
            return ord(self.symbol[0]) < ord(other.symbol[0])
        else:
            return self.prob < other.prob

    def has_children(self):
        return self.left or self.right

    def __str__(self, level=0) -> str:
        indent = "  " * level
        result = f"{indent}Node(s: {self.symbol}, p: {round(self.prob, 2)})\n"

        if self.left is not None:
            result += f"{indent} {bcolors.OKGREEN}L -> {bcolors.ENDC}{self.left.__str__(level + 1)}"
        if self.right is not None:
            result += f"{indent} {bcolors.WARNING}R -> {bcolors.ENDC}{self.right.__str__(level + 1)}"

        return result

In [33]:
class Huffman:
    def __generate_char_freq_and_prob__(self, text):
        """Generate char frequence and probability from input text"""
        char_freq = {}
        length = len(text)

        # char frequence
        for c in text:
            char_freq[c] = char_freq.get(c, 0) + 1

        # char probability
        char_prob = {c: freq / length for c, freq in char_freq.items()}
        return char_freq, char_prob

    def __create_probability_queue__(self, char_prob: dict):
        """Create priority queue based on character probabilities"""
        prio_queue = PriorityQueue()
        for c in char_prob.keys():
            prio_queue.put(Node(c, char_prob[c]))

        return prio_queue

    def __create_tree__(self, prio_queue: PriorityQueue[Node]) -> Node:
        """Create Huffman tree nodes"""
        # while the queue contains more than one element (root)
        while prio_queue.qsize() > 1:
            left = prio_queue.get()
            right = prio_queue.get()

            # For each element a Node is created and a code 0 or 1 is given
            left.codeword = "0"
            right.codeword = "1"

            # the two nodes extracted from queue are inserted as a new node
            new_node = Node(
                left.symbol + right.symbol, left.prob + right.prob, left, right
            )
            prio_queue.put(new_node)

        return prio_queue.get()

    def __make_codes__(self, node: Node, code="") -> dict:
        """Generate Huffman codes by traversing the tree"""
        if node is None:
            return {}

        # If node is a leaf then return code
        if not node.has_children():
            return {node.symbol: code}

        # Otherwise continue with depth first recursion
        codes = {}
        codes.update(self.__make_codes__(node.left, code + "0"))
        codes.update(self.__make_codes__(node.right, code + "1"))
        return codes

    def __init__(self, text: str):
        if not text:
            raise ValueError("Input text cannot be empty.")

        self.char_freq, self.char_prob = self.__generate_char_freq_and_prob__(text)
        self.queue = self.__create_probability_queue__(self.char_prob)
        self.tree_root = self.__create_tree__(self.queue)
        self.code = self.__make_codes__(self.tree_root)

    def status(self):
        print(f"Frequencies: {self.char_freq}")
        print("Probabilities: ", {k: round(v, 2) for k, v in self.char_prob.items()})
        print(f"Tree: \n{self.tree_root}")
        print(f"Codes: {self.code}")

    def encode(self, text: str) -> str:
        """Encode provided input text using Huffman coding"""
        if not self.code:
            raise ValueError("Huffman codes must be generated before encoding.")

        output = []
        for symbol in text:
            if symbol not in self.code:
                raise ValueError(f"Symbol '{symbol}' not found in encoding dictionary.")
            output.append(self.code[symbol])

        encoded_string = "".join(output)

        return encoded_string

    def decode(self, encoded_text: str) -> str:
        """Decode the encoded text using Huffman tree"""
        if not self.tree_root:
            raise ValueError("Huffman tree must be generated before decoding.")

        decoded_string = []
        current_node = self.tree_root

        for bit in encoded_text:
            # Traverse the tree based on each bit in encoded_text until it reaches a leaf
            if bit == "1":
                current_node = current_node.right
            elif bit == "0":
                current_node = current_node.left

            # In case of leaf we add the decoded symbol
            if current_node.left is None and current_node.right is None:
                decoded_string.append(current_node.symbol)
                # Then goes back to root to continue with decoding process
                current_node = self.tree_root

        return "".join(decoded_string)

### Example

In [38]:
input_text = "labellalilliballa"
print(f"Input text: {input_text}")

s = Huffman(input_text)
s.status()

coded_text = s.encode(input_text)
print(f"Coded input: {coded_text}")

original_text = s.decode(coded_text)
print(f"Original input: {original_text}")

print(f"Encoded in {len(coded_text)} bits")
print(f"Compression ratio is {len(coded_text)/(len(input_text)*8)*100:.2f} %")
assert input_text == original_text, "Error in encoding and decoding"

Input text: labellalilliballa
Frequencies: {'l': 8, 'a': 4, 'b': 2, 'e': 1, 'i': 2}
Probabilities:  {'l': 0.47, 'a': 0.24, 'b': 0.12, 'e': 0.06, 'i': 0.12}
Tree: 
Node(s: laieb, p: 1.0)
 L ->   Node(s: l, p: 0.47)
 R ->   Node(s: aieb, p: 0.53)
   L ->     Node(s: a, p: 0.24)
   R ->     Node(s: ieb, p: 0.29)
     L ->       Node(s: i, p: 0.12)
     R ->       Node(s: eb, p: 0.18)
       L ->         Node(s: e, p: 0.06)
       R ->         Node(s: b, p: 0.12)

Codes: {'l': '0', 'a': '10', 'i': '110', 'e': '1110', 'b': '1111'}
Coded input: 0101111111000100110001101111100010
Original input: labellalilliballa
Encoded in 34 bits
Compression ratio is 25.00 %
